In [292]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from datetime import date
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [293]:
# train = pd.read_csv('data/train.csv')
# oil = pd.read_csv('data/oil.csv')
# holiday = pd.read_csv('data/holidays_events.csv')
# stores = pd.read_csv('data/stores.csv')
# transactions = pd.read_csv('data/transactions.csv')

In [294]:
train = pd.read_csv('data/train.csv')
train['date'] = pd.to_datetime(train.date)

In [295]:
train.shape

(3000888, 6)

In [296]:
3000888 * 70 / 100

2100621.6

In [297]:
3000888 - 2100621

900267

In [298]:
train_split = train[0:2100621]
validation = train[2100621:]

In [299]:
def groupedDf(train):
    dfgr = train.groupby(['store_nbr', 'family']).rolling(window=120, on='date')['sales'].mean().bfill().reset_index()
    dfgr.rename(columns={'sales': 'sales_avg30'}, inplace=True)

    return pd.merge(train, dfgr, how='left')

In [300]:
df_merged = groupedDf(train_split)

In [301]:
# train.groupby(['store_nbr', 'family']).rolling(window=30, on='date')['sales'].mean().bfill().reset_index()

In [302]:
# df_merged

In [303]:
pred = df_merged[['store_nbr', 'family', 'sales_avg30']][df_merged['date'].dt.date == date(2016, 3, 26)] #select latest day for future validation

In [304]:
pred

,store_nbr,family,sales_avg30
2097414,1,AUTOMOTIVE,4.725000
2097415,1,BABY CARE,0.000000
2097416,1,BEAUTY,2.191667
2097417,1,BEVERAGES,2010.758333
2097418,1,BOOKS,0.000000
...,...,...,...
2099191,9,POULTRY,486.160633
2099192,9,PREPARED FOODS,98.717667
2099193,9,PRODUCE,1682.488257
2099194,9,SCHOOL AND OFFICE SUPPLIES,4.800000


In [305]:
result = validation.merge(pred, how='left')

In [306]:
result.head()

,id,date,store_nbr,family,sales,onpromotion,sales_avg30
0,2100621,2016-03-27,49,CELEBRATION,32.000,0,38.650000
1,2100622,2016-03-27,49,CLEANING,1977.000,26,1930.558333
2,2100623,2016-03-27,49,DAIRY,3036.000,0,2464.441667
3,2100624,2016-03-27,49,DELI,519.661,0,397.231683
4,2100625,2016-03-27,49,EGGS,583.000,0,483.950000


In [307]:
print(f'MSE: {np.sqrt(mean_squared_error(result.sales, result.sales_avg30)):.2f}')
print(f'MSLE: {np.sqrt(mean_squared_log_error(result.sales, result.sales_avg30)):.2f}')

MSE: 509.72
MSLE: 0.76


In [308]:
test = pd.read_csv('data/test.csv')
test['date'] = pd.to_datetime(test.date)

In [309]:
df_merged = groupedDf(train)

In [310]:
y_pred = df_merged[['store_nbr', 'family', 'sales_avg30']][df_merged['date'].dt.date == date(2017, 8, 15)]

In [311]:
# y_pred

In [312]:
result = pd.merge(test, y_pred, how='left')

In [313]:
# result

In [314]:
result = result[['id', 'sales_avg30']].rename(columns={'sales_avg30': 'sales'})

In [315]:
result.to_csv('submission.csv', index=False)

In [316]:
# train = pd.read_csv('data/train.csv')
# oil = pd.read_csv('data/oil.csv')
# holiday = pd.read_csv('data/holidays_events.csv')
# stores = pd.read_csv('data/stores.csv')
# transactions = pd.read_csv('data/transactions.csv')

In [317]:
train = pd.read_csv('data/train.csv')
train['date'] = pd.to_datetime(train.date)

In [318]:
train.shape

(3000888, 6)

In [319]:
3000888 * 70 / 100

2100621.6

In [320]:
3000888 - 2100621

900267

In [321]:
train_split = train[0:2100621]
validation = train[2100621:]

In [ ]:
def groupedDf(train):
    dfgr = train.groupby(['store_nbr', 'family']).rolling(window=30, on='date')['sales'].mean().bfill().reset_index()
    dfgr['sales'] = dfgr.sales.ewm(span=30).mean()
    dfgr.rename(columns={'sales': 'sales_avg30'}, inplace=True)

    return pd.merge(train, dfgr, how='left')

In [323]:
df_merged = groupedDf(train_split)

TypeError: object of type 'ExponentialMovingWindow' has no len()

In [ ]:
# train.groupby(['store_nbr', 'family']).rolling(window=30, on='date')['sales'].mean().bfill().reset_index()

In [ ]:
# df_merged

In [ ]:
pred = df_merged[['store_nbr', 'family', 'sales_avg30']][df_merged['date'].dt.date == date(2016, 3, 26)] #select latest day for future validation

In [ ]:
pred

,store_nbr,family,sales_avg30
2097414,1,AUTOMOTIVE,5.277439e+00
2097415,1,BABY CARE,4.026230e-34
2097416,1,BEAUTY,2.192108e+00
2097417,1,BEVERAGES,2.025768e+03
2097418,1,BOOKS,1.536637e-31
...,...,...,...
2099191,9,POULTRY,4.722999e+02
2099192,9,PREPARED FOODS,1.048030e+02
2099193,9,PRODUCE,1.684952e+03
2099194,9,SCHOOL AND OFFICE SUPPLIES,5.930474e+00


In [ ]:
result = validation.merge(pred, how='left')

In [ ]:
result.head()

,id,date,store_nbr,family,sales,onpromotion,sales_avg30
0,2100621,2016-03-27,49,CELEBRATION,32.000,0,39.828798
1,2100622,2016-03-27,49,CLEANING,1977.000,26,1802.567125
2,2100623,2016-03-27,49,DAIRY,3036.000,0,2467.262093
3,2100624,2016-03-27,49,DELI,519.661,0,387.391415
4,2100625,2016-03-27,49,EGGS,583.000,0,475.906927


In [ ]:
print(f'MSE: {np.sqrt(mean_squared_error(result.sales, result.sales_avg30)):.2f}')
print(f'MSLE: {np.sqrt(mean_squared_log_error(result.sales, result.sales_avg30)):.2f}')

MSE: 515.73
MSLE: 0.75


In [ ]:
test = pd.read_csv('data/test.csv')
test['date'] = pd.to_datetime(test.date)

In [ ]:
df_merged = groupedDf(train)

In [ ]:
y_pred = df_merged[['store_nbr', 'family', 'sales_avg30']][df_merged['date'].dt.date == date(2017, 8, 15)]

In [ ]:
# y_pred

In [ ]:
result = pd.merge(test, y_pred, how='left')

In [ ]:
# result

In [ ]:
result = result[['id', 'sales_avg30']].rename(columns={'sales_avg30': 'sales'})

In [ ]:
result.to_csv('submission.csv', index=False)